In [1]:
# TODO: What to do wit the split_int variable?
# so far set to 1, but implies a specific center?
# so far the waterfall only comes from the first of the 5 models in the ensemble
# DONE: Adjust the scaling to the UKB scale
# choose a theme from https://www.gradio.app/docs/gradio/themes
# add disclaimer that the results are not to be used for clinical purposes!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [2]:
#%load_ext autoreload
#%autoreload 2
#%pip install gradio
#%pip install shap

import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ast
from pipeline import * #Load our package with classes pipeline, models, pp (preprocessing), plot, and more
path= userpath(os.environ.get("USER", os.environ.get("USERNAME")), project="hcc") # Choose your own project here, only works if you added specific project in user_settings.json

Debug: userpath called with user=phkoop, project=hcc
Debug: Loaded user_settings.json: {'phkoop': {'hcc': '/Users/phkoop/Library/CloudStorage/OneDrive-UniklinikRWTHAachen/drive/public/projects/hcc', 'public_online': '/Users/phkoop/Library/CloudStorage/OneDrive-UniklinikRWTHAachen/drive/public', 'drive_disc': '', 'modeling_pipeline': '/Users/phkoop/Documents/GitHub/modeling_pipeline/src/modeling_pipeline'}, 'janni': {'hcc': 'C:/Users/janni/Uniklinik RWTH Aachen/CRC-1382-A11 - public/projects/hcc', 'facs_aml': 'C:/Users/janni/OneDrive/Dokumente/PostDoc/Data/FACS_AML'}, 'pkoop': {'hcc': 'C:/Users/pkoop/OneDrive - Uniklinik RWTH Aachen/drive/public/projects/hcc'}, 'felix': {'cca': 'C:/Users/felix/OneDrive - Uniklinik RWTH Aachen/public/projects/cca'}, 'david': {'hcc': '...'}}
Debug: Found path for user phkoop and project hcc: /Users/phkoop/Library/CloudStorage/OneDrive-UniklinikRWTHAachen/drive/public/projects/hcc


In [3]:
import matplotlib as plttt
plttt.__version__

'3.7.1'

In [4]:
pl=load_Pipeline('./.app_temp/orig_plot/model/Pipeline_HCC_all_Model_TOP15_RFC_external_val.joblib')

In [5]:
#TODO: encoding UKB to AOU SEX?
encodings={'SEX': {'female':0,'male':1,'else':0}} ##### to be discussed!
#replace the value of a column of a dataframe by a dictionary


In [6]:
## CHange file path to reduce columns and Adjust_AOU_UKB_MinMax Manual Adjustements.xlsx"
f1 = os.path.join("./.app_temp/orig_plot/model/reduce_columns.xlsx")
f2 = os.path.join("./.app_temp/orig_plot/model/Adjust_AOU_UKB_MinMax Manual Adjustements.xlsx")
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2)
model = "top15"

In [7]:
pred_val_df=pd.read_csv('./.app_temp/orig_plot/model/pred_values_top15.csv')
pred_val_df.__len__()

93623

In [8]:
variables = df1.Feature[df1[model]==1].unique()
variable_names = list(df1[df1[model]==True]["Feature"].unique())


units = df2.set_index("column")[["Unit_ukb","Unit_aou"]].copy() #get both units from file . Important UKB first!!! 
units.fillna("",inplace=True) # replace nans 

units_dict = units.apply(lambda row: list(row), axis=1).to_dict() #create dict for units 
units_factor_dict  = df2.set_index("column")["adjust_unit"].copy() .to_dict() #create dict for unit factor 

# Example data: variable names and corresponding units
def random_AUC_plot(filename ):
    # Generate random data for false positive rate (FPR) and true positive rate (TPR)
    fpr = np.linspace(0, 1, 100)  # FPR values from 0 to 1
    tpr = np.random.uniform(0, 1, 100)  # Random TPR values between 0 and 1

    # Sort TPR values to make it look like a ROC curve
    tpr = np.sort(tpr)

    # Plot the random ROC-like curve
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, label='Random ROC-like Curve')
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess (AUC=0.5)')  # Diagonal line for reference
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Random Data)')
    plt.legend(loc='lower right')

    # Save the plot to a file
    plot_filename = filename 
    plt.savefig(plot_filename)
    plt.close()



# Function to update the unit for a specific variable when its button is clicked
def update_unit(variable):
    # Get the units for the selected variable
    units = units_dict[variable]
    
    # Get the current index and update it
    index = current_unit_index[variable]
    new_index = (index + 1) % len(units)
    current_unit = units[new_index]
    
    # Increment the index and wrap around if needed
    current_unit_index[variable] = new_index
    
    return current_unit

def process_variables(variables):
    df=pd.DataFrame(variables.values(), index=variables.keys()).T
    df=df[sorted(df.columns, key=lambda x: [i.replace('remainder__','') for i in pl.ohe.feature_names_in_.tolist()].index(x))]
    df['split_int']=[1]

    # replace the data by the encodings
    for key in encodings.keys():
        if key in df.columns:
            df[key]=df[key].map(encodings[key])
    df=df.astype(float)

    # scale the data -> bisher nur für einzelne Values
    def scale_minmax(column,x):
        i=df2[df2.column==column]
        return (x-i.min_ukb)/(i.max_ukb-i.min_ukb)

    df_scaled=df.copy()
    for col in df_scaled.columns:
        try:
            df_scaled[col]=[scale_minmax(col,df[col].values[0]).values[0]]
        except:
            print(f"Could not scale {col}")
            df_scaled[col]=df[col][0]

    # create the waterfall plot and the prediction
    def plot_waterfall(df=df_scaled, pl=pl):
        plt.close('all')
        idx=0
        X=df
        for estimator in pl.trained_model.models:
            estimator=estimator[0].best_estimator_

            explainer = shap.explainers.TreeExplainer(estimator)
            sv = explainer(X)
            exp = shap.Explanation(sv.values[:,:,1], 
                            sv.base_values[:,1], 
                            data=X.values, 
                            feature_names=X.columns,)
            ax=shap.plots.waterfall(exp[idx],show=False)
            plt.tight_layout()
            os.makedirs('./.app_temp/', exist_ok=True)
            plt.savefig('./.app_temp/waterfall.png', bbox_inches='tight')
            return './.app_temp/waterfall.png'
    plot_waterfall=plot_waterfall(df_scaled, pl)

    def violin_plot_w_hline(output_path:str='./.app_temp/violin.png',prediction:float=0.5):
        pred_val_df=pd.read_csv('./.app_temp/orig_plot/model/pred_values_top15.csv')
        plt.close('all')
        fig,ax =plot.create_violin_plot(pl,X_val=pred_val_df,y_val=pred_val_df,model=pl.master_RFC,ohe=pl.ohe, gap=-0.1, width=0.8, thresholds_choice=[0,.35,.55,1],y_pred=pred_val_df.y_pred,save_fig=False)
        ax.axhline(y=prediction, color='r', linestyle='--', linewidth=2)
        plt.title('')
        fig.savefig(output_path)
        return output_path


    prediction=pl.master_RFC.predict_proba(pl.ohe.transform(df_scaled))[0]

    plot_violin =violin_plot_w_hline(output_path='./.app_temp/violin.png', prediction=prediction)


    return f'Input:\n{variables}\nScaled:\n{dict(df_scaled)}', round(prediction,ndigits=2),  plot_waterfall, plot_violin



def apply_factors(vars,variable_names,current_unit_index):
    factored_vars = []
    for var,name in zip(vars,variable_names):
        unit_index = current_unit_index[name]
        if unit_index: # ==1 -> All of us -> use factor to transform into ukb units 
            factor = units_factor_dict[name]
            var_new = (lambda x : eval(factor))(var)
        else:
            var_new = var
        factored_vars.append(var_new)
    return factored_vars


# Gradio interface function
def create_interface(current_unit_index):
    
    # Define a Blocks context
    with gr.Blocks(css=".align-bottom { display: flex; align-items: flex-end; }",theme='default') as demo:
        size_image=100
        #gr.Image("/Users/phkoop/Library/CloudStorage/OneDrive-UniklinikRWTHAachen/drive/public/orga/Presentations/Schneider_lab_logo_round_with_text.png",width=size_image,height=size_image,show_label=False,show_download_button=False,show_fullscreen_button=False,show_share_button=False)
        gr.Markdown(f'<center>Try our new Model for the prediction of HCC risk.\nThe prediction is based on the following training Pipeline: {pl.name}</center>')
        gr.Markdown(f'<p align="center">\n<b>Disclaimer:</b><br>\nThis tool has been developed solely for demonstration purposes and is not intended for use as a medical device under any circumstances. It is not designed to diagnose, treat, or manage any medical condition. The predictions generated by this tool should not be used as a basis for any medical or clinical decisions. The authors and developers disclaim any responsibility or liability for decisions or actions influenced by the use of this tool. Users are strongly advised to consult with qualified healthcare professionals for any medical concerns or decisions.\n</p>')
        gr.Markdown("<center>The Pipeline was developed by JC, PHK and TS in the Group of Prof. Carolin Schneider at RWTH Aachen University.</center>")
        
        # Create a row for inputs
        with gr.Column():
            vars = []
            for variable in variable_names:
                with gr.Row(elem_classes="align-bottom"):  # Group input and button in a row
                    if variable in units_dict.keys():
                        initial_unit = units_dict[variable][0]
                        if len(list(set(units_dict[variable]))) > 1:
                            with gr.Column(scale=2):
                                # Create a number input for each variable
                                var = gr.Number(label=f"{variable}", interactive=True)
                            with gr.Column(scale=1):
                                unit_button = gr.Button(f"{initial_unit}")
                                # Set up the button to update the unit when clicked
                                unit_button.click(fn=update_unit, inputs=gr.State(variable), outputs=unit_button)
                        else:
                            var = gr.Number(label=f"{variable}"+(f"[{initial_unit}]" if initial_unit != "" else ""), interactive=True)
                    else:
                        choices = ["True", "False"] if variable != "SEX" else ["male","female","else"]
                        var = gr.Dropdown(label=f"{variable}", choices = choices)   
                vars.append(var)
            # Button to trigger the calculation and generate output
            submit_button = gr.Button("Generate Output")
            
        # Outputs
        gr.Markdown(f'<p align="center">\n<b>Disclaimer:</b><br>\nThis tool has been developed solely for demonstration purposes and is not intended for use as a medical device under any circumstances. It is not designed to diagnose, treat, or manage any medical condition. The predictions generated by this tool should not be used as a basis for any medical or clinical decisions. The authors and developers disclaim any responsibility or liability for decisions or actions influenced by the use of this tool. Users are strongly advised to consult with qualified healthcare professionals for any medical concerns or decisions.\n\n</p>')
        output_text = gr.Text(label="Showing the chosen parameters")
        output_value_1 = gr.Number(label="Prediction Value")
        #output_value_2 = gr.Number(label="Metric 2")
        output_plot2=gr.Image(label="Violin plot of the predictions on the internal validation cohort from the UKBiobank, with the individual prediction as the red dashed line.")
        gr.Markdown(f'*Violin plot of the prediction values of the UKBiobank (n={pred_val_df.__len__()}) internal validation cohort, with the individual prediction displayed as the red dashed line.\n *')
        output_plot1 = gr.Image(label="Waterfall plot based on the SHAP-value estimation")
        gr.Markdown('*Waterfall plot based on the SHAP-value estimation. The plot shows the contribution of each feature to the overall prediction. The base value is the average prediction of the model. The prediction is the sum of the base value and the contributions of the contributing features. The plot is based on the first model of the ensemble classifier.*')
        
    
        def on_submit(*vars):
            variables = {}
            vars_factored = apply_factors(vars,variable_names,current_unit_index)
            for var_names,var in zip(variable_names,vars_factored):
                try:
                    variables[var_names] = ast.literal_eval(var)
                except:
                    variables[var_names] = var
            return process_variables(variables)
        
        # Set up the button to trigger the output generation
        submit_button.click(fn=on_submit, inputs=vars, outputs=[output_text,output_value_1,output_plot1,output_plot2])#,output_value_2,output_plot])
    return demo

# Track the current index of the unit for each variable
current_unit_index = {var: 0 for var in variable_names}
# Launching the app
create_interface(current_unit_index).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b77d0fb0d5bb64788f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
